In [1]:
# 1. 라이브러리 설치
%pip install transformers accelerate timm pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 316.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 295.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 255.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 453.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 254.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# 2. 라이브러리 import
import torch
from transformers import AutoProcessor, InternVLForConditionalGeneration
from PIL import Image

In [3]:
# 3. Colab 환경에서 GPU 사용 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"디바이스 확인: {device}")

디바이스 확인: cuda


In [4]:
# Colab에서 입력창으로 Hugging Face Token 받기 
import os

from getpass import getpass
hf_token = getpass('🔑 Enter your Hugging Face API token: ')

# 환경 변수로 저장 (코드에 노출 안됨)
os.environ['HUGGINGFACE_TOKEN'] = hf_token
print("✅ Hugging Face token saved to environment variable.")

# huggingface_hub 로그인
from huggingface_hub import login
login(hf_token)

🔑 Enter your Hugging Face API token:  ········


✅ Hugging Face token saved to environment variable.


In [5]:
# 4. 모델 로드 (InternVL3-8B, Qwen2.5-7B)
# 일단 8B 모델 사용 더 좋은 GPU 있으면 파라미터 하나 더 높은거 써봐도 좋을듯..
# 모델 로드 시 환경변수에서 토큰 꺼내기
model_name = "OpenGVLab/InternVL3-8B-hf"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
token = os.environ['HUGGINGFACE_TOKEN']

processor = AutoProcessor.from_pretrained(model_name, use_auth_token=token)
model = InternVLForConditionalGeneration.from_pretrained(model_name, use_auth_token=token).to(device)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/processing_auto.py:254: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


processor_config.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/481 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/126 [00:00<?, ?B/s]

In [6]:
print(model)

InternVLForConditionalGeneration(
  (model): InternVLModel(
    (vision_tower): InternVLVisionModel(
      (embeddings): InternVLVisionEmbeddings(
        (patch_embeddings): InternVLVisionPatchEmbeddings(
          (projection): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14))
        )
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (encoder): InternVLVisionEncoder(
        (layer): ModuleList(
          (0-23): 24 x InternVLVisionLayer(
            (attention): InternVLVisionAttention(
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (projection_layer): Linear(in_features=1024, out_features=1024, bias=True)
              (projection_dropout): Identity()
              (q_norm): Identity()
              (k_norm): Identity()
            )
            (mlp): InternVLV

In [9]:
from transformers import AutoProcessor, AutoModelForImageTextToText
import gc
import os
import torch

# 모델 및 processor 로드 (float16)
processor = AutoProcessor.from_pretrained(model_name)
model = AutoModelForImageTextToText.from_pretrained(
    model_name,
    device_map=device,
    torch_dtype=torch.float16  # ✅ float16로 메모리 절약
).to(device)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [21]:
import json
image_folder = "./car_images"
output_json_file = "car_descriptions.json"
prompt = (
   "As an automotive designer, briefly and precisely summarize this car's design: body type, proportions, surface, lighting, grill, wheels, color, unique/futuristic elements."
)

results = []

for idx, img_file in enumerate(sorted(os.listdir(image_folder))):
    if not img_file.lower().endswith((".png", ".jpg", ".jpeg", ".webp")):
        continue

    img_path = os.path.join(image_folder, img_file)
    messages = [[
        {
            "role": "user",
            "content": [
                {"type": "image", "path": img_path},
                {"type": "text", "text": prompt},
            ],
        }
    ]]

    try:
        # processor로 inputs 생성 (float16로 맞춤)
        inputs = processor.apply_chat_template(
            messages,
            padding=True,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt"
        ).to(device, dtype=torch.float16)

        # no_grad로 gradient 계산 끔 (추론용 메모리 절약)
        with torch.no_grad():
            output = model.generate(**inputs, max_new_tokens=300)

        # 결과 디코딩
        decoded_outputs = processor.batch_decode(output, skip_special_tokens=True)
        description = decoded_outputs[0]

        # assistant 부분만 저장
        if "assistant" in description:
            description_clean = description.split("assistant", 1)[-1].strip()
        else:
            description_clean = description.strip()


        #  output 확인
        print(f"\n✅ [{idx + 1}] {img_file}")
        print(f"{'-'*60}")
        print(description_clean)
        print(f"{'-'*60}")
        
        results.append({
            "image": img_file,
            "description": description_clean
        })


    except Exception as e:
        print(f"⚠️ [{idx + 1}] {img_file} 처리 중 오류: {e}")

    # 메모리 비우기 (PyTorch + Python GC)
    del inputs, output, decoded_outputs
    torch.cuda.empty_cache()
    gc.collect()

# 모든 결과를 JSON 파일로 저장
with open(output_json_file, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"✅ 모든 결과가 '{output_json_file}'에 저장되었습니다.")


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.



✅ [1] 10_2026_캐스퍼.png
------------------------------------------------------------
This SUV features a compact body type with a balanced, modern proportion. The surface is smooth with subtle curves, enhancing aerodynamics. It has sleek, integrated lighting. The grill is narrow and horizontal, contributing to a contemporary look. The wheels are stylish with a multi-spoke design. The color is a matte olive green, giving it a rugged yet sophisticated appearance. Unique elements include roof rails and a slightly elevated stance, adding a touch of futurism.
------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.



✅ [2] 10_유니버스_수소전기버스.png
------------------------------------------------------------
This bus features a modern, elongated body type with a streamlined, aerodynamic design. The proportions are balanced, with a long, low profile. The surface is smooth with large, tinted windows for passenger comfort. The lighting includes sleek, integrated headlights and LED strips. The grill is minimalistic and wide, enhancing the bus's modern look. It has large, multi-spoke alloy wheels. The color is predominantly white, giving it a clean appearance. Unique elements include the futuristic, curved front and advanced lighting design.
------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


...


✅ [55] 9_2026_캐스퍼_일렉트릭.png
------------------------------------------------------------
This SUV features a modern, compact body type with balanced proportions. The design includes smooth, flowing surfaces and sleek, integrated lighting. The grill is prominent and horizontally oriented, complementing the contemporary aesthetic. It has stylish alloy wheels and a neutral color palette. Unique elements include subtle angular lines and a slightly elevated stance, giving it a futuristic yet practical look.
------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.



✅ [56] 9_스타리아_라운지_리무진_Hybrid.png
------------------------------------------------------------
This car features a modern minivan body type with a sleek, elongated proportions. The surface is smooth with tinted windows, giving it a sophisticated look. It has subtle lighting integrated into the body. The grill is understated, blending with the front design. The wheels are stylish and alloy-finished. The color is a glossy black, enhancing its premium feel. Unique elements include the streamlined side profile and potential futuristic LED accents.
------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.



✅ [57] 9_싼타페_Hybrid.png
------------------------------------------------------------
This SUV features a modern, boxy body type with a balanced proportion. The surface is smooth with subtle lines for aerodynamics. The lighting includes sleek, integrated headlights and taillights. The grill is prominent and horizontal, adding to the bold front fascia. The wheels are large with a multi-spoke alloy design. The color is a neutral silver-gray. Unique elements include the roof rails and a slightly elevated stance, giving it a rugged yet contemporary look.
------------------------------------------------------------

✅ [58] 9_유니버스.png
------------------------------------------------------------
This bus features a modern, elongated body type with a streamlined, aerodynamic design. The proportions are balanced, with a long, low profile. The surface is smooth with minimal panel lines, enhancing its sleek appearance. Large, tinted windows run along the side, providing ample visibility. The ligh